In [2]:
## IMPORTACIÓN GENERAL DE LIBRERIAS.
import re 
import sys
import json
import nltk
import geocoder
import requests
import warnings
import descartes

import numpy as np
import pandas as pd
import seaborn as sns
import networkx as nx
import datetime as DT
import category_encoders as ce
import matplotlib.pyplot as plt

# SKLEARN.
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

# RANDOM FOREST.
from urllib.request import urlopen
from shapely.geometry import Point, Polygon
from sklearn.ensemble import RandomForestRegressor

# XGBOOST.
from xgboost import XGBClassifier
import xgboost as xgb

In [3]:
#NLTK: https://www.nltk.org
from langdetect import detect
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.tokenize import TweetTokenizer
from nltk.stem.wordnet import WordNetLemmatizer

#WORDCLOUD
from PIL import Image
from wordcloud import WordCloud

# CONFIGURACIÓN.
%matplotlib inline
plt.style.use('default')
pd.options.display.float_format = '{:20,.2f}'.format
warnings.filterwarnings('ignore')
sns.set(style="whitegrid") 
plt.rcParams['figure.figsize'] = (15, 10)
np.set_printoptions(threshold=sys.maxsize)

In [4]:
import re
import json
import requests
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split 
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.preprocessing import StandardScaler
from nltk.stem.snowball import SnowballStemmer
from string import punctuation
from textblob import TextBlob

from nltk.tokenize import sent_tokenize, word_tokenize, regexp_tokenize # funtions for standard tokenisation
from nltk.tokenize import TweetTokenizer # function for tweets tokenization
from nltk.corpus import stopwords
import string

from nltk.util import bigrams

In [5]:
#Limpia puntuacion, quita también usuarios y hashtags (@ y #)
def clean_tweet(words):
    result = []
    for word in words:
        stripped_word = word.strip()
        if ((stripped_word.isalnum() == True) and (not(stripped_word.isdigit()))):      
            result.append(stripped_word)
    
    return result

def filtrarPalabras(miArray):
    variable = ''
    for key in miArray:
         if '#' in str(key):
            variable = variable + ' ' + str(key)
    return variable

In [6]:
tknzr = TweetTokenizer() # initialization of Tweet Tokenizer

def mean_words_length(text):
    words = word_tokenize(text)
    word_lengths = [len(w) for w in words]
    return round(np.mean(word_lengths),1)

In [7]:
def clean(tweet): 
           
    # Urls
    tweet = re.sub(r"https?:\/\/t.co\/[A-Za-z0-9]+", "", tweet)
        
    # Words with punctuations and special characters
    punctuations = '@#!?+&*[]-%.:/();$=><|{}^' + "'`"
    for p in punctuations:
        tweet = tweet.replace(p, f' {p} ')
        
    # ... and ..
    tweet = tweet.replace('...', ' ... ')
    if '...' not in tweet:
        tweet = tweet.replace('..', ' ... ') 
    
    return tweet

### =====================================================================
### LECTURA DE CSV.
### =====================================================================

In [8]:
#Importacion del archivo CSV de fuente
#https://www.kaggle.com/c/nlp-getting-started
original_train = pd.read_csv('../data/train.csv')
original_test = pd.read_csv('../data/test.csv')
original_sample_submission = pd.read_csv('../data/sample_submission.csv')

### =====================================================================
### PROCESAMIENTO DE DATOS.
### =====================================================================

In [9]:
#COMPLETAMOS LOS NULOS CON UN TEXTO: 'VACIO'
original_train.fillna('vacio', inplace = True)
original_test.fillna('vacio', inplace = True)

In [10]:
original_train.head()

,id,keyword,location,text,target
0,1,vacio,vacio,Our Deeds are the Reason of this #earthquake M...,1
1,4,vacio,vacio,Forest fire near La Ronge Sask. Canada,1
2,5,vacio,vacio,All residents asked to 'shelter in place' are ...,1
3,6,vacio,vacio,"13,000 people receive #wildfires evacuation or...",1
4,7,vacio,vacio,Just got sent this photo from Ruby #Alaska as ...,1


In [9]:
original_train['text'] = original_train['text'].apply(lambda s : clean(s))
original_test['text'] = original_test['text'].apply(lambda s : clean(s))

In [10]:
#PASAMOS TODO A MINÚSCULAS (TRAIN).
original_train['text'] = original_train['text'].str.lower()
original_train['location'] = original_train['location'].str.lower()
original_train['keyword'] = original_train['keyword'].str.lower()
#PASAMOS TODO A MINÚSCULAS (TEST).
original_test['text'] = original_test['text'].str.lower()
original_test['location'] = original_test['location'].str.lower()
original_test['keyword'] = original_test['keyword'].str.lower()

In [11]:
## ==================================================
## TRATAMIENTO INCLUYENDO STOP WORDS.
## ==================================================

In [12]:
#LONGITUD DEL TWEET Y CANTIDAD DE PALABRAS (TRAIN).
original_train['length'] = original_train['text'].str.len()
original_train['totalwords'] = original_train['text'].str.split().str.len()
original_train['words'] = original_train.text.str.strip().str.split()
#LONGITUD DEL TWEET Y CANTIDAD DE PALABRAS (TEST).
original_test['length'] = original_test['text'].str.len()
original_test['totalwords'] = original_test['text'].str.split().str.len()
original_test['words'] = original_test.text.str.strip().str.split()

In [13]:
#ARMANDO UNA LISTA DE PALABRAS QUE PUEDEN REPRESENTAR CATÁSTROFES (TRAIN & TEST).
selected_words_singular=['fire','flood','inundate','earthquake','quake','deluge','euption','twister','tornado','hurricane', 'landslide','typhoon','wildfire','forest fire','drought','avalanche','urgent','important','danger','warrning','evacuation']
selected_words_plural=['fires','floods', 'earthquakes','quakes','deluges','rashes','tornadoes','hurricanes', 'landslides','typhoons','wildfires','forest fires','droughts','avalanches']
selected_words_other=['heat wave','died','flooding','flooded','damage','urgent','important','danger','warrning','help','evacuation']
col_one_list = original_train['keyword'].tolist()
selected_words = selected_words_singular + selected_words_plural + selected_words_other + col_one_list
s = set(selected_words)

In [14]:
#BUSCAMOS LAS COINCIDENCIAS CON PALABRAS CLAVES Y SEPARAMOS LOS HASHTAGS (TRAIN).
original_train = original_train.assign(hashtags=[filtrarPalabras(el) for el in original_train.words])
original_train = original_train.assign(matches=[len(set(el) & s) for el in original_train.words])
#BUSCAMOS LAS COINCIDENCIAS CON PALABRAS CLAVES Y SEPARAMOS LOS HASHTAGS (TEST).
original_test = original_test.assign(hashtags=[filtrarPalabras(el) for el in original_test.words])
original_test = original_test.assign(matches=[len(set(el) & s) for el in original_test.words])

In [15]:
#CONTAMOS LA CANTIDAD DE HASHTAGS (TRAIN).
original_train['preguntas'] = original_train['text'].str.count('[!]') + original_train['text'].str.count('[?]')
original_train['simbolos'] = original_train['text'].str.count('[!]') + original_train['text'].str.count('[?]') + original_train['text'].str.count('=') + original_train['text'].str.count('>')
#CONTAMOS LA CANTIDAD DE HASHTAGS (TEST).
original_test['preguntas'] = original_test['text'].str.count('[!]') + original_test['text'].str.count('[?]')
original_test['simbolos'] = original_test['text'].str.count('[!]') + original_test['text'].str.count('[?]') + original_test['text'].str.count('=') + original_test['text'].str.count('>')

In [16]:
#MARCAMOS QUIENES USAN UBICACIÓN Y PALABRAS CLAVES (TRAIN).
original_train['conUbicacion'] = 0
original_train['conKeyword'] = 0
original_train.loc[original_train['location'] != 'vacio', ['conUbicacion']] = 1
original_train.loc[original_train['keyword'] != 'vacio', ['conKeyword']] = 1
#MARCAMOS QUIENES USAN UBICACIÓN Y PALABRAS CLAVES (TEST).
original_test['conUbicacion'] = 0
original_test['conKeyword'] = 0
original_test.loc[original_test['location'] != 'vacio', ['conUbicacion']] = 1
original_test.loc[original_test['keyword'] != 'vacio', ['conKeyword']] = 1

In [17]:
# original_train['text'] = original_train['text'].apply(lambda s : clean(s))
# original_test['text'] = original_test['text'].apply(lambda s : clean(s))

In [18]:
## ==================================================
## TRATAMIENTO QUITANDO LAS STOP WORDS.
## ==================================================

In [19]:
# Tratamiento del set con nltk y nlpaug
# NLTK
# Sabiendo que todos los tweets son en idioma ingles, quitamos las stopwords
stop_words = set(stopwords.words('english'))
original_train['text'] =  original_train['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
original_test['text'] =  original_test['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

In [20]:
#PASAMOS A LA RAÍZ IDOMÁTICA DE LA PALABRA.
stemmer = SnowballStemmer('english')
original_train['text'] = original_train['text'].apply(lambda x:' '.join([stemmer.stem(y) for y in x.split()]))
original_test['text'] = original_test['text'].apply(lambda x:' '.join([stemmer.stem(y) for y in x.split()]))

In [21]:
#LONGITUD DEL TWEET Y CANTIDAD DE PALABRAS (TRAIN).
original_train['length_SW'] = original_train['text'].str.len()
original_train['totalwords_SW'] = original_train['text'].str.split().str.len()
original_train['lenXword_SW'] = (original_train['length_SW'] / original_train['totalwords_SW'])
original_train['stopWords'] = (original_train['totalwords'] - original_train['totalwords_SW']) + 1
original_train['totalXsw'] = (original_train['totalwords_SW'] / original_train['stopWords'])
#LONGITUD DEL TWEET Y CANTIDAD DE PALABRAS (TEST).
original_test['length_SW'] = original_test['text'].str.len()
original_test['totalwords_SW'] = original_test['text'].str.split().str.len()
original_test['lenXword_SW'] = (original_test['length_SW'] / original_test['totalwords_SW'])
original_test['stopWords'] = (original_test['totalwords'] - original_test['totalwords_SW']) + 1
original_test['totalXsw'] = (original_test['totalwords_SW'] / original_test['stopWords'])

In [22]:
numbers_regex = r"(\d+\.?,?\s?\d+)"
hashtags_regex = r"#\w+"
mentions_regex = r"@\w+"
punctuation_regex = r"[^\w\s]"

In [23]:
# words count
original_train['words_count'] = original_train['text'].apply(lambda x: len(tknzr.tokenize(x)))
# numbers count
original_train['numbers_count'] = original_train['text'].apply(lambda x: len(regexp_tokenize(x, numbers_regex)))
# hashtags count
original_train['hashtags_count'] = original_train['text'].apply(lambda x: len(regexp_tokenize(x, hashtags_regex)))
# mentions count
original_train['mentions_count'] = original_train['text'].apply(lambda x: len(regexp_tokenize(x, mentions_regex)))
# mean words length
original_train['mean_words_length'] = original_train['text'].apply(mean_words_length)
# mean words length
original_train['characters_count'] = original_train['text'].apply(lambda x: len(x))
# lowercase tokens
original_train['lowercase_bag_o_w'] = original_train['text'].apply(lambda x: [w for w in tknzr.tokenize(x.lower())])
# stopwords
original_train['stopwords'] = original_train['lowercase_bag_o_w'].apply(lambda x: [t for t in x if t in stopwords.words('english')])
# stopwords count
original_train['stopwords_count'] = original_train['stopwords'].apply(lambda x: len(x))
# alpha words only (excludes mentions and hashtags)
original_train['alpha_only'] = original_train['lowercase_bag_o_w'].apply(lambda x: [t for t in x if t.isalpha()])
# counts of alpha words only
original_train['alpha_count'] = original_train['alpha_only'].apply(lambda x: len(x))
# counts of punctuation marks only
original_train['punctuation_count'] = original_train['text'].apply(lambda x: len(regexp_tokenize(x, punctuation_regex)))

In [24]:
# words count
original_test['words_count'] = original_test['text'].apply(lambda x: len(tknzr.tokenize(x)))
# numbers count
original_test['numbers_count'] = original_test['text'].apply(lambda x: len(regexp_tokenize(x, numbers_regex)))
# hashtags count
original_test['hashtags_count'] = original_test['text'].apply(lambda x: len(regexp_tokenize(x, hashtags_regex)))
# mentions count
original_test['mentions_count'] = original_test['text'].apply(lambda x: len(regexp_tokenize(x, mentions_regex)))
# mean words length
original_test['mean_words_length'] = original_test['text'].apply(mean_words_length)
# mean words length
original_test['characters_count'] = original_test['text'].apply(lambda x: len(x))
# lowercase tokens
original_test['lowercase_bag_o_w'] = original_test['text'].apply(lambda x: [w for w in tknzr.tokenize(x.lower())])
# stopwords
original_test['stopwords'] = original_test['lowercase_bag_o_w'].apply(lambda x: [t for t in x if t in stopwords.words('english')])
# stopwords count
original_test['stopwords_count'] = original_test['stopwords'].apply(lambda x: len(x))
# alpha words only (excludes mentions and hashtags)
original_test['alpha_only'] = original_test['lowercase_bag_o_w'].apply(lambda x: [t for t in x if t.isalpha()])
# counts of alpha words only
original_test['alpha_count'] = original_test['alpha_only'].apply(lambda x: len(x))
# counts of punctuation marks only
original_test['punctuation_count'] = original_test['text'].apply(lambda x: len(regexp_tokenize(x, punctuation_regex)))

In [25]:
#QUITAMOS LOS NÚMEROS.
original_train['text'] = original_train['text'].str.replace('\d+', '')
original_test['text'] = original_test['text'].str.replace('\d+', '')
original_train['text'] = original_train['text'].str.replace('vacio', '')
original_test['text'] = original_test['text'].str.replace('vacio', '')

In [26]:
## ==================================================
## ENCODING
## ==================================================

In [27]:
#HOT ENCODING PARA KEYWORD (TRAIN).
dummies = pd.get_dummies(original_train['keyword'], drop_first=False)
original_train = pd.concat([original_train, dummies], axis=1)
original_train.drop('keyword', 1, inplace = True)

#HOT ENCODING PARA KEYWORD (TEST).
dummies = pd.get_dummies(original_test['keyword'], drop_first=False)
original_test = pd.concat([original_test, dummies], axis=1)
original_test.drop('keyword', 1, inplace = True)

In [28]:
# encoder = ce.BinaryEncoder(cols=['keyword'])
# original_train = encoder.fit_transform(original_train)
# original_train.info()

In [29]:
# encoder = ce.BinaryEncoder(cols=['keyword'])
# original_test = encoder.fit_transform(original_test)
# original_test.info()

In [30]:
original_train = original_train.fillna(0)
original_test = original_test.fillna(0)

In [31]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import twitter_samples, stopwords
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from nltk import FreqDist, classify, NaiveBayesClassifier
import nltk
import re, string, random

In [32]:
def remove_noise(tweet_tokens, stop_words = ()):

    cleaned_tokens = []

    for token, tag in pos_tag(tweet_tokens):
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
        token = re.sub("(@[A-Za-z0-9_]+)","", token)

        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'

        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)

        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens

def get_all_words(cleaned_tokens_list):
    for tokens in cleaned_tokens_list:
        for token in tokens:
            yield token

def get_tweets_for_model(cleaned_tokens_list):
    for tweet_tokens in cleaned_tokens_list:
        yield dict([token, True] for token in tweet_tokens)

In [33]:
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('twitter_samples')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /home/andres/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/andres/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package twitter_samples to
[nltk_data]     /home/andres/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package punkt to /home/andres/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/andres/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [34]:
    positive_tweets = twitter_samples.strings('positive_tweets.json')
    negative_tweets = twitter_samples.strings('negative_tweets.json')
    text = twitter_samples.strings('tweets.20150430-223406.json')
    tweet_tokens = twitter_samples.tokenized('positive_tweets.json')[0]

    stop_words = stopwords.words('english')

    positive_tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
    negative_tweet_tokens = twitter_samples.tokenized('negative_tweets.json')

    positive_cleaned_tokens_list = []
    negative_cleaned_tokens_list = []

    for tokens in positive_tweet_tokens:
        positive_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

    for tokens in negative_tweet_tokens:
        negative_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

    all_pos_words = get_all_words(positive_cleaned_tokens_list)

    freq_dist_pos = FreqDist(all_pos_words)
    print(freq_dist_pos.most_common(10))

    positive_tokens_for_model = get_tweets_for_model(positive_cleaned_tokens_list)
    negative_tokens_for_model = get_tweets_for_model(negative_cleaned_tokens_list)

    positive_dataset = [(tweet_dict, "Positive")
                         for tweet_dict in positive_tokens_for_model]

    negative_dataset = [(tweet_dict, "Negative")
                         for tweet_dict in negative_tokens_for_model]

    dataset = positive_dataset + negative_dataset

    random.shuffle(dataset)

    train_data = dataset[:7000]
    test_data = dataset[7000:]

    classifier = NaiveBayesClassifier.train(train_data)

[(':)', 3691), (':-)', 701), (':d', 658), ('thanks', 388), ('follow', 357), ('love', 333), ('...', 290), ('good', 283), ('get', 263), ('thank', 253)]


In [35]:
original_train['token_limpios'] = original_train['text'].apply(lambda s : remove_noise(word_tokenize(s)))
original_train['sentimiento'] = original_train['token_limpios'].apply(lambda s: classifier.classify(dict([token, True] for token in s)))

In [36]:
original_test['token_limpios'] = original_test['text'].apply(lambda s : remove_noise(word_tokenize(s)))
original_test['sentimiento'] = original_test['token_limpios'].apply(lambda s: classifier.classify(dict([token, True] for token in s)))

In [37]:
#HOT ENCODING PARA SENTIMIENTO (TRAIN).
dummies = pd.get_dummies(original_train['sentimiento'], drop_first=False)
original_train = pd.concat([original_train, dummies], axis=1)
original_train.drop('sentimiento', 1, inplace = True)
original_train.drop('token_limpios', 1, inplace = True)

#HOT ENCODING PARA SENTIMIENTO (TEST).
dummies = pd.get_dummies(original_test['sentimiento'], drop_first=False)
original_test = pd.concat([original_test, dummies], axis=1)
original_test.drop('sentimiento', 1, inplace = True)
original_test.drop('token_limpios', 1, inplace = True)

In [38]:
## ==================================================
## DEJAMOS SOLO LAS COLUMNAS NUMÉRICAS.
## ==================================================

In [39]:
#ELIMINAMOS LAS COLUMNAS QUE NO SON NUMÉRICAS (TRAIN).
original_train.drop('text', 1, inplace = True)
original_train.drop('words', 1, inplace = True)
original_train.drop('location', 1, inplace = True)
original_train.drop('hashtags', 1, inplace = True)
#ELIMINAMOS LAS COLUMNAS QUE NO SON NUMÉRICAS (TEST).
original_test.drop('text', 1, inplace = True)
original_test.drop('words', 1, inplace = True)
original_test.drop('location', 1, inplace = True)
original_test.drop('hashtags', 1, inplace = True)

In [40]:
original_train.drop(['lowercase_bag_o_w','stopwords','alpha_only'], axis=1, inplace= True)
original_test.drop(['lowercase_bag_o_w','stopwords','alpha_only'], axis=1, inplace= True)

### =====================================================================
### RESGUARDAMOS LOS DATOS EN CSV.
### =====================================================================

In [41]:
#Vemos la estructura del dataframe TRAIN.
original_train.head()

,id,target,length,totalwords,matches,preguntas,simbolos,conUbicacion,conKeyword,length_SW,...,wild%20fires,wildfire,windstorm,wounded,wounds,wreck,wreckage,wrecked,Negative,Positive
0,1,1,71,14,1,0,0,0,0,43,...,0,0,0,0,0,0,0,0,0,1
1,4,1,40,8,1,0,0,0,0,38,...,0,0,0,0,0,0,0,0,0,1
2,5,1,139,25,1,0,0,0,0,75,...,0,0,0,0,0,0,0,0,0,1
3,6,1,67,9,2,0,0,0,0,52,...,0,0,0,0,0,0,0,0,1,0
4,7,1,92,18,2,0,0,0,0,56,...,0,0,0,0,0,0,0,0,1,0


In [42]:
original_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Columns: 247 entries, id to Positive
dtypes: float64(3), int64(20), uint8(224)
memory usage: 3.0 MB


In [43]:
#Vemos la estructura del dataframe TEST.
original_test.head()

,id,length,totalwords,matches,preguntas,simbolos,conUbicacion,conKeyword,length_SW,totalwords_SW,...,wild%20fires,wildfire,windstorm,wounded,wounds,wreck,wreckage,wrecked,Negative,Positive
0,0,34,6,1,0,0,0,0,24,4,...,0,0,0,0,0,0,0,0,1,0
1,2,68,11,1,0,0,0,0,52,9,...,0,0,0,0,0,0,0,0,0,1
2,3,96,19,1,0,0,0,0,59,10,...,0,0,0,0,0,0,0,0,1,0
3,9,46,7,2,0,0,0,0,36,7,...,0,0,0,0,0,0,0,0,0,1
4,11,45,8,1,0,0,0,0,37,6,...,0,0,0,0,0,0,0,0,1,0


In [44]:
original_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3263 entries, 0 to 3262
Columns: 246 entries, id to Positive
dtypes: float64(3), int64(19), uint8(224)
memory usage: 1.2 MB


In [45]:
#Vemos la estructura del dataframe SAMPLE_SUBMISSION.
original_sample_submission.head()

,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0


In [46]:
original_train.to_csv('../data/processed/original_train.csv',index=False)
original_test.to_csv('../data/processed/original_test.csv',index=False)

### =====================================================================
### FIN.
### =====================================================================